In [3]:
import pandas as pd
df=pd.read_csv("datasets/housing/insurance.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/housing/insurance.csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
!pip install plotly matplotlib seaborn --quiet 
# ! tells Jupyter to run the command in the system shell (not Python) and quiet installs the packages silently without showing progress bars or logs.

In [ ]:
import plotly.express as px
import plotly.io as pio #Imports the Plotly Input/Output module as pio
pio.renderers.default = 'notebook'  # or 'colab', 'iframe', 'browser'
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

%matplotlib inline - tells matplotlib to display the plots inline inside the notebook(right below the code cell)

In [ ]:
fig=px.histogram(df,x='bmi',
                 marginal='box', #used for above box plot in the following figure
                 color_discrete_sequence=['red'],
                 title='Distribution of bmi')
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
fig=px.histogram(df,x='charges',
                 marginal='box',color='smoker',
                 color_discrete_sequence=['red','yellow'],
                 title='Distribution of charges with somkers highlighted')
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
fig=px.histogram(df,x='age',
                 marginal='box',color='smoker',
                 color_discrete_sequence=['red','yellow'],
                 title='Distribution of age with somkers highlighted')
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
df.value_counts()

In [ ]:
df.smoker.value_counts()

In [ ]:
px.histogram(df,x='smoker',color='sex',title='Smokers')

In [ ]:
fig=px.scatter(df,x='age',y='charges',color='smoker',opacity=0.5,hover_data=['sex'],title='age vs charges')
fig.update_traces(marker_size=5)
fig.show()
#hover- When you hover your mouse over a data point, it shows additional info from the DataFrame — in this case, the sex of the person.
#update_traces-it sets the size of each marker (dot) to 5.
#result-three clusters are visible

In [ ]:
fig=px.scatter(df,x='bmi',y='charges',color='smoker',opacity=0.5,hover_data=['sex'],title='age vs charges')
fig.update_traces(marker_size=5)
fig.show()

Since in scattter plot points gets badly overlapped so to get more clear distibution use violin for getting the density

In [ ]:
fig=px.violin(df,x='smoker',y='charges',hover_data=['sex'],title='age vs charges')
fig.update_traces(marker_size=5)
fig.show()

Now calculating the correlation of 'charges' feature with other features.
The more is the corr absolute value , stronger is the correlation.
Negative correlation implies inverse relation(with 'charges').
More is the negative correlation , more stronger is the inverse dependence.  
Note: The .corr() method only works with numerical features.

In [ ]:
df.charges.corr(df.age)

In [ ]:
df.charges.corr(df.bmi)

In [ ]:
df.charges.corr(df.children)

In [ ]:
smoker_val={'no':0,'yes':1}
df['smoker_numeric']=df.smoker.map(smoker_val)
df.smoker_numeric

In [ ]:
df.charges.corr(df.smoker_numeric)

In [ ]:
#df.corr()-will give error if any column of non numeric entries is there
df.select_dtypes(include='number').corr()


In [ ]:
sns.heatmap(df.select_dtypes(include='number').corr(),cmap='Reds',annot=True)
plt.title('Correlation Matrix')

In [ ]:
non_smoker_df=df[df.smoker=='no']

In [ ]:
plt.title('Age vs charges')
sns.scatterplot(data=non_smoker_df, x='age',y='charges',alpha=0.7,s=15);
#alpha==opacity,s-size of the mark

Manually fitting a linear model.

In [ ]:
def estimate_charge(age,w,b):
    return w*age+b

In [ ]:
ages=non_smoker_df.age

In [ ]:
ages

In [ ]:
w=40
b=100
estimate=estimate_charge(ages,w,b)
estimate

In [ ]:
a=non_smoker_df.charges
a

In [ ]:
plt.plot(ages,estimate,'r-')
plt.xlabel('Ages')
plt.ylabel('Estimated charges')
#r-red colour, +->plused line ,using-instead will give a solid line.

Estimated Charges (Line) vs Actual Charges (Scatter)

In [ ]:
target = non_smoker_df.charges
plt.plot(ages,estimate,'r',alpha=0.8)
plt.scatter(ages,target,s=8,alpha=0.9) #s-size
plt.xlabel('Ages')
plt.ylabel('Charges')
plt.legend(['Estimate','Actual']) #estimate-1st plot command, actual-2nd plot command

In [ ]:
def tryparameters(w,b):
    ages=non_smoker_df.age
    target=non_smoker_df.charges
    estimate=estimate_charge(ages,w,b)
    plt.plot(ages,estimate,'r',alpha=0.9)
    plt.scatter(ages,target,s=8,alpha=0.8)
    plt.xlabel('Age')
    plt.ylabel('Charges')
    plt.legend(['Estimate','Actual'])

In [ ]:
tryparameters(200,-1200)

In [ ]:
import numpy as np


In [ ]:
target=non_smoker_df.charges
prediction=estimate_charge(non_smoker_df.age,200,-1200)

In [ ]:
def rmse(target,prediction):
    return np.sqrt(np.mean(np.square(target-prediction)))

In [ ]:
rmse(target,prediction)
#=> on average , each prediction differs the actual by 5071

In [ ]:
def parameters(w,b):
    ages=non_smoker_df.age
    target=non_smoker_df.charges
    estimate=estimate_charge(ages,w,b)
    plt.plot(ages,estimate,'r',alpha=0.9)
    plt.scatter(ages,target,s=8,alpha=0.8)
    plt.xlabel('Age')
    plt.ylabel('Charges')
    plt.legend(['Estimate','Actual'])
    loss=rmse(target,estimate)
    print("RMSE loss:",loss)

In [ ]:
parameters(350,-4000)

Using Scikit-Learn to Recreate the Above Implementation

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
help(model.fit)

In [ ]:
inputs=non_smoker_df[['age']]
target=non_smoker_df.charges
print("inputs shape:",inputs.shape)
print("target shape:",target.shape)

In [ ]:
model.fit(inputs,target)

When fitting the linear regression model using Scikit-learn's LinearRegression(), the above default parameters are applied.

In [ ]:
model.predict([[23],
               [21],
               [74]])

While applying the model on new data, above warning appeared indicating that the input does not have valid feature names. This occurs when the model is trained using a DataFrame (with named columns), but the new data is passed as a raw list or array. To resolve this, the new input should also be structured as a DataFrame with the same feature names used during training, ensuring correct feature alignment.

In [ ]:
predictions =model.predict(inputs)
predictions

In [ ]:
target

In [ ]:
rmse(target,predictions)
#this too much rmse below is due to outliers otherwise on other data points the model is pretty good

In [ ]:
#w 
model.coef_


In [ ]:
#b
model.intercept_

In [ ]:
tryparameters(model.coef_,model.intercept_)

Evaluating RMSE with 'age' and 'bmi' as inputs.

In [ ]:
inputs = non_smoker_df[['age','bmi']]
target = non_smoker_df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))
#there is very less difference in rmse even after adding bmi becuase correlation of bmi and charges was very less

In [ ]:
model.coef_ , model.intercept_

Finding correlation of 'charges' with 'children' and including it in inputs.

In [ ]:
non_smoker_df.charges.corr(non_smoker_df.children)

In [ ]:
fig=px.strip(non_smoker_df,x='children',y='charges',title='Children vs charges')
fig.update_traces(marker_size=4,marker_opacity=0.4)
fig.show()

In [ ]:
inputs = non_smoker_df[['age','bmi','children']]
target = non_smoker_df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))

Above RMSE is calculated for non_smoker_df but now applying it for whole dataset df.

In [ ]:
inputs = df[['age','bmi','children']] #now applying for whole input data including smokers also
target = df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))

Huge error is observed while taking the same input but for whole dataset.  
It is so becuase it learned patterns only from non-smoker data.  
Applying it to whole data(including smoker data also) lead to poor predictions.

In [ ]:
smoker_code=df.smoker_numeric
smoker_code

In [ ]:
df

In [ ]:
type(df)

In [ ]:
df.charges.corr(df.smoker_numeric)

In [ ]:
inputs = non_smoker_df[['age','bmi','children','smoker_numeric']]
target = non_smoker_df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))

In [ ]:
sns.barplot(data=df,x='sex',y='charges')

In [ ]:
sex_numeric={'female':0,'male':1} #binary categorisation
df['sex_numeric']=df.sex.map(sex_numeric)

In [ ]:
df.sex_numeric

In [ ]:
df.charges.corr(df.sex_numeric)

In [ ]:
df

In [ ]:
inputs = df[['age','bmi','children','smoker_numeric','sex_numeric']]
target = df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))

In [ ]:
#one hot encoding for region
sns.barplot(data=df,x='region',y='charges')

In [ ]:
from sklearn import preprocessing
enc=preprocessing.OneHotEncoder() #onehotencoder expects a dataframe(2D)
enc.fit(df[['region']]) #using single square bracket will give a series(1D) and not a dataframe
enc.categories_

In [ ]:
enc.transform([['northeast'],
               ['northwest']]).toarray()

In [ ]:
one_hot=enc.transform(df[['region']]).toarray()
one_hot

In [ ]:
df[['northeast','northwest','southeast','southwest']]=one_hot

In [ ]:
df

In [ ]:
inputs = df[['age','bmi','children','smoker_numeric','sex_numeric','northeast','northwest','southeast','southwest']]
target = df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))

In [ ]:
inputs.loc[10] #10th index row specifications

In [ ]:
model.predict([[20,22.4,0,0,0,1,0,0,0]])

In [ ]:
inp_clms=['age','bmi','children','smoker_numeric','sex_numeric','northeast','northwest','southeast','southwest']
weights_df=pd.DataFrame({
    'feature':np.append(inp_clms,1),
    'weights':np.append(model.coef_,model.intercept_)
})
weights_df

Weights of different features not always imply which features are important because ranges of different features are different. As age can go upto 60 so it can create (60×256) effect but northeast can take max value 1 hence can effect (1×587) only.So to avoid it we need to rescale the data of features with larger range.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
numeric_cols=['age','bmi','children']
scaler=StandardScaler()
scaler.fit(df[numeric_cols])

In [ ]:
scaler.mean_

In [ ]:
scaler.var_

In [ ]:
scaled_numeric=scaler.transform(df[numeric_cols])
scaled_numeric

In [ ]:
cat_colms=['smoker_numeric','sex_numeric','northeast','northwest','southeast','southwest']
categorical_data=df[cat_colms].values

In [ ]:
inputs.iloc[0]

In [ ]:
#final_inputs = np.hstack([scaled_numeric, categorical_data])
final_inputs=np.concatenate((scaled_numeric,categorical_data),axis=1)

In [ ]:
final_inputs[0]

In [ ]:
targets=df.charges
model=LinearRegression().fit(final_inputs,targets)
predictions=model.predict(final_inputs)
loss=rmse(targets,predictions)
print("Loss:",loss)

In [ ]:
weights_df=pd.DataFrame({
    'feature':np.append(numeric_cols+cat_colms,1),
    'weights':np.append(model.coef_,model.intercept_)
})
weights_df.sort_values('weights',ascending=False)
#1 represents the intercept

In [ ]:
new=[[20,22.4,0,0,0,1,0,0,0]]

In [ ]:
scaler.transform([[20,22.4,0]])

In [ ]:
model.predict([[-1.36756307, -1.35556464, -0.90861367,0,0,1,0,0,0]])

Since now , I solved for non smokers which contibutes to the maximum of the dataset .
Now let's try for smokers.

In [ ]:
smoker_df=df[df.smoker=='yes']

In [ ]:
ages=smoker_df.age
ages

In [ ]:
print(smoker_df.charges)

In [ ]:
target = smoker_df.charges
plt.scatter(ages,target,s=8,alpha=0.9) #s-size
plt.xlabel('Ages')
plt.ylabel('Charges')
#plt.legend(['Estimate','Actual']) estimate-1st plot command, actual-2nd plot command

In [ ]:
def parameters(w,b):
    ages=smoker_df.age
    target=smoker_df.charges
    estimate=estimate_charge(ages,w,b)
    plt.plot(ages,estimate,'r',alpha=0.9)
    plt.scatter(ages,target,s=8,alpha=0.8)
    plt.xlabel('Age')
    plt.ylabel('Charges')
    plt.legend(['Estimate','Actual'])
    loss=rmse(target,estimate)
    print("RMSE loss:",loss)

In [ ]:
parameters(350,16000)

Since it is difficult to fit the data giving the parameters manually. Hence using sklearn with LinersRegression is the better option.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()

In [ ]:
inputs=smoker_df[['age']]
target=smoker_df.charges
print("inputs shape:",inputs.shape)
print("target shape:",target.shape)

In [ ]:
model.fit(inputs,target)

In [ ]:
predictions =model.predict(inputs)
predictions

In [ ]:
target

In [ ]:
rmse(target,predictions)

In [ ]:
inputs = smoker_df[['age','bmi']]
target = smoker_df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))


In [ ]:
# Correlation of all numeric features in smoker_df with 'charges'
correlations = smoker_df.corr(numeric_only=True)['charges'].sort_values(ascending=False)
print(correlations)


In [ ]:
# Correlation of all numeric features in non_smoker_df with 'charges'
correlations = non_smoker_df.corr(numeric_only=True)['charges'].sort_values(ascending=False)
print(correlations)


BMI plays major role in smoker_df but not in non_smoker_df.

In [ ]:
inputs = smoker_df[['age','bmi','southeast']]
target = smoker_df['charges']
model=LinearRegression().fit(inputs,target)
prediction=model.predict(inputs)
print("LOss:",rmse(target,prediction))

In [ ]:
def evaluate_features_one_by_one(df, feature_list, target_col):
    print("Evaluating feature combinations:\n")
    selected_features = []

    for feature in feature_list:
        selected_features.append(feature)

        inputs = df[selected_features]
        target = df[target_col]

        model = LinearRegression().fit(inputs, target)
        prediction = model.predict(inputs)

        loss = rmse(target, prediction)
        print(f"Features: {selected_features} --> RMSE: {loss:.2f}")
features = ['bmi', 'age', 'southeast', 'sex_numeric', 'children','southwest','northwest','northeast']
evaluate_features_one_by_one(smoker_df, features, target_col='charges')

In [ ]:
non_smoker_df=df[df.smoker=='no']

In [ ]:
def evaluate_features_one_by_one(df, feature_list, target_col):
    print("Evaluating feature combinations:\n")
    selected_features = []

    for feature in feature_list:
        selected_features.append(feature)

        inputs = df[selected_features]
        target = df[target_col]

        model = LinearRegression().fit(inputs, target)
        prediction = model.predict(inputs)

        loss = rmse(target, prediction)
        print(f"Features: {selected_features} --> RMSE: {loss:.2f}")
features = ['bmi', 'age', 'southeast', 'sex_numeric', 'children','southwest','northwest','northeast']
evaluate_features_one_by_one(non_smoker_df, features, target_col='charges')

In [ ]:
def evaluate_features_one_by_one(df, feature_list, target_col):
    print("Evaluating feature combinations:\n")
    selected_features = []

    for feature in feature_list:
        selected_features.append(feature)

        inputs = df[selected_features]
        target = df[target_col]

        model = LinearRegression().fit(inputs, target)
        prediction = model.predict(inputs)

        loss = rmse(target, prediction)
        print(f"Features: {selected_features} --> RMSE: {loss:.2f}")
features = ['bmi', 'age', 'southeast', 'sex_numeric', 'children','southwest','northwest','northeast']
evaluate_features_one_by_one(df, features, target_col='charges')

As giving combined dataset(df) including smoker and non smoker gives more error hence it is better to give separate inputs in the form of smoker_df and 
non_smoker_df.